### 【問題1】スクラッチを振り返る 
ここまでのスクラッチを振り返り、ディープラーニングを実装するためにはどのようなものが必要だったかを列挙してください。

・重み、バイアスの初期化、更新
・エポックのループ
・活性化関数を通して出力する。
・forward,back propagationを繰り返す。

### 【問題2】スクラッチとTensorFlowの対応を考える 
以下のサンプルコードを見て、先ほど列挙した「ディープラーニングを実装するために必要なもの」が
TensorFlowではどう実装されているかを確認してください。
それを簡単に言葉でまとめてください。単純な一対一の対応であるとは限りません。

In [1]:
"""
TensorFlowで実装したニューラルネットワークを使いIrisデータセットを2値分類する
"""
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf

# データセットの読み込み
#dataset_path = r"Iris.csv"
df = pd.read_csv(r"C:\Users\anai\dive\Dataset\iris\iris.csv")
# データフレームから条件抽出
df = df[(df["Species"] == "Iris-versicolor")|(df["Species"] == "Iris-virginica")]
y = df["Species"]
X = df.loc[:, ["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]]
y = np.array(y)
X = np.array(X)
# ラベルを数値に変換
y[y=='Iris-versicolor'] = 0
y[y=='Iris-virginica'] = 1
y = y.astype(np.int)[:, np.newaxis]

# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

class GetMiniBatch:
    """
    ミニバッチを取得するイテレータ

    Parameters
    ----------
    X : 次の形のndarray, shape (n_samples, n_features)
      学習データ
    y : 次の形のndarray, shape (n_samples, 1)
      正解値
    batch_size : int
      バッチサイズ
    seed : int
      NumPyの乱数のシード
    """
    def __init__(self, X, y, batch_size = 10, seed=0):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self.X = X[shuffle_index]
        self.y = y[shuffle_index]
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)
    def __len__(self):
        return self._stop
    def __getitem__(self,item):
        p0 = item*self.batch_size
        p1 = item*self.batch_size + self.batch_size
        return self.X[p0:p1], self.y[p0:p1]        
    def __iter__(self):
        self._counter = 0
        return self
    def __next__(self):
        if self._counter >= self._stop:
            raise StopIteration()
        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self.X[p0:p1], self.y[p0:p1]

# ハイパーパラメータの設定
learning_rate = 0.01
batch_size = 10
num_epochs = 10

n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 1

# 計算グラフに渡す引数の形を決める
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])

# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)

def example_net(x):
    """
    単純な3層ニューラルネットワーク
    """
    # 重みとバイアスの宣言
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }

    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] # tf.addと+は等価である
    return layer_output

# ネットワーク構造の読み込み                               
logits = example_net(X)

# 目的関数
loss_op = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=Y, logits=logits))
# 最適化手法
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# 推定結果
correct_pred = tf.equal(tf.sign(Y - 0.5), tf.sign(tf.sigmoid(logits) - 0.5))
# 指標値計算
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# variableの初期化
init = tf.global_variables_initializer()

# 計算グラフの実行
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        # エポックごとにループ
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)
        total_loss = 0
        total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # ミニバッチごとにループ
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
            total_acc += acc
        total_loss /= n_samples
        total_acc /= n_samples
        val_loss, val_acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}, val_acc : {:.3f}".format(epoch, loss, val_loss, acc, val_acc))
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
    print("test_acc : {:.3f}".format(test_acc))

C:\Users\anai\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\anai\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\anai\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\anai\AppData\Roaming\Python\Python36\site-pack

Epoch 0, loss : 29.8439, val_loss : 16.0679, acc : 0.250, val_acc : 0.625
Epoch 1, loss : 0.0000, val_loss : 0.3342, acc : 1.000, val_acc : 0.875
Epoch 2, loss : 0.0000, val_loss : 0.4121, acc : 1.000, val_acc : 0.875
Epoch 3, loss : 0.0025, val_loss : 1.7410, acc : 1.000, val_acc : 0.812
Epoch 4, loss : 0.0000, val_loss : 1.0760, acc : 1.000, val_acc : 0.875
Epoch 5, loss : 0.0000, val_loss : 0.2701, acc : 1.000, val_acc : 0.938
Epoch 6, loss : 0.0000, val_loss : 0.0118, acc : 1.000, val_acc : 1.000
Epoch 7, loss : 0.0000, val_loss : 0.0002, acc : 1.000, val_acc : 1.000
Epoch 8, loss : 0.0000, val_loss : 1.1928, acc : 1.000, val_acc : 0.875
Epoch 9, loss : 0.0000, val_loss : 1.4731, acc : 1.000, val_acc : 0.812
test_acc : 0.800


・初期化は　tf.global_variables_initializer()　一文で完了
　更新は　tf.train.AdamOptimizer(learning_rate=learning_rate)　一文で完了
・エポックのループは同じ
・活性化関数は　tf.nn.relu(layer_1)　一文で完了
・forward,backward propagationそれぞれの記述がない。

### 【問題3】3種類全ての目的変数を使用したIrisのモデルを作成 
Irisデータセットのtrain.csvの中で、目的変数Speciesに含まれる3種類全てを分類できるモデルを作成してください。

In [2]:
"""
TensorFlowで実装したニューラルネットワークを使いIrisデータセットを3値分類する
"""
from sklearn.preprocessing import OneHotEncoder
import sklearn.preprocessing as sp

# データセットの読み込み
#dataset_path = r"Iris.csv"
df = pd.read_csv(r"C:\Users\anai\dive\Dataset\iris\iris.csv", delimiter=',')
# データフレームから条件抽出
#df = df[(df["Species"] == "Iris-versicolor")|(df["Species"] == "Iris-virginica")]
y = df['Species']
X = df.loc[:, ["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]]
y = np.array(y)
X = np.array(X)

# ラベルを数値に変換
y[y=='Iris-setosa'] = 0
y[y=='Iris-versicolor'] = 1
y[y=='Iris-virginica'] = 2

#onehot
#enc = OneHotEncoder(handle_unknown='ignore', sparse=False, dtype='int', categories='auto')
enc = OneHotEncoder(handle_unknown='ignore', sparse=False, dtype='int')
y_1hot = enc.fit_transform(y[:, np.newaxis])

# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y_1hot, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

# ハイパーパラメータの設定
learning_rate = 0.01
batch_size = 10
num_epochs = 10

n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 3

# 計算グラフに渡す引数の形を決める
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])

def example_net(x):
    """
    単純な3層ニューラルネットワーク
    """

    # 重みとバイアスの宣言
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }

    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] # tf.addと+は等価である
    return layer_output

# ネットワーク構造の読み込み                               
logits = example_net(X)

# 目的関数
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=logits))
# 最適化手法
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# 推定結果
correct_pred = tf.equal(tf.argmax(Y,1), tf.argmax(tf.nn.softmax(logits),1))
# 指標値計算
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# variableの初期化
init = tf.global_variables_initializer()

get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)

# 計算グラフの実行
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        # エポックごとにループ
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)
        total_loss = 0
        total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # ミニバッチごとにループ
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
            total_acc += acc
        total_loss /= n_samples
        total_acc /= n_samples
        val_loss, val_acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}, val_acc : {:.3f}".format(epoch, loss, val_loss, acc, val_acc))
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
    print("test_acc : {:.3f}".format(test_acc))

W0905 16:03:05.635721 17340 deprecation.py:323] From <ipython-input-2-796d9595a7e9>:78: softmax_cross_entropy_with_logits (from tensorflow.python.ops.nn_ops) is deprecated and will be removed in a future version.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



Epoch 0, loss : 72.7798, val_loss : 36.1441, acc : 0.333, val_acc : 0.708
Epoch 1, loss : 11.9029, val_loss : 38.1278, acc : 0.833, val_acc : 0.625
Epoch 2, loss : 14.8873, val_loss : 8.6922, acc : 0.500, val_acc : 0.750
Epoch 3, loss : 0.0000, val_loss : 7.8215, acc : 1.000, val_acc : 0.708
Epoch 4, loss : 0.0005, val_loss : 3.4804, acc : 1.000, val_acc : 0.875
Epoch 5, loss : 2.0249, val_loss : 3.6416, acc : 0.667, val_acc : 0.875
Epoch 6, loss : 0.2180, val_loss : 3.7833, acc : 0.833, val_acc : 0.833
Epoch 7, loss : 0.0000, val_loss : 3.4388, acc : 1.000, val_acc : 0.875
Epoch 8, loss : 0.0000, val_loss : 2.5853, acc : 1.000, val_acc : 0.875
Epoch 9, loss : 1.1831, val_loss : 3.7219, acc : 0.833, val_acc : 0.875
test_acc : 0.733


### 【問題4】House Pricesのモデルを作成 
回帰問題のデータセットであるHouse Pricesを使用したモデルを作成してください。
House Prices: Advanced Regression Techniques
この中のtrain.csvをダウンロードし、目的変数としてSalePrice、説明変数として、GrLivAreaとYearBuiltを使ってください。
説明変数はさらに増やしても構いません。分類問題と回帰問題の違いを考慮してください。

In [17]:
"""
TensorFlowで実装したニューラルネットワークを使いHouse Pricesデータセットの価格を推定する。
"""
from sklearn.preprocessing import StandardScaler

# データセットの読み込み
df = pd.read_csv(r"C:\Users\anai\dive\Dataset\House Prices\train.csv", delimiter=',')
#df_test = pd.read_csv(r"C:\Users\anai\dive\Dataset\House Prices\test.csv", delimiter=',')

# データフレームから条件抽出
y = df['SalePrice']
X = df.loc[:,["GrLivArea", "YearBuilt"]]

y = np.array(y)
X = np.array(X)

# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

#std = StandardScaler()
#X_train = std.fit_transform(X_train)
#X_test = std.fit_transform(X_test)
#X_val = std.fit_transform(X_val)

y_test = y_test.reshape(-1, 1)
y_train = y_train.reshape(-1, 1)
y_val = y_val.reshape(-1, 1)

# ハイパーパラメータの設定
learning_rate = 0.00000000001
batch_size = 10
num_epochs = 10

n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 1

# 計算グラフに渡す引数の形を決める
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])

def example_net(x):
    """
    単純な3層ニューラルネットワーク
    """
    # 重みとバイアスの宣言
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes])),
        
        'w4': tf.Variable(tf.random_normal([n_input, n_classes])),
        'w5': tf.Variable(tf.random_normal([n_hidden1, n_classes]))

    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }

    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] # tf.addと+は等価である
    
    #layer_output = tf.add(tf.matmul(x, weights['w4']), biases['b3'])
    #layer_output = tf.add(tf.matmul(layer_1, weights['w5']), biases['b3'])
    return layer_output

# ネットワーク構造の読み込み                               
logits = example_net(X)
# 目的関数
loss_op = tf.reduce_mean(tf.losses.mean_squared_error(labels=Y, predictions=logits))
#loss_op = tf.losses.mean_squared_error(labels=Y, predictions=logits)

# 最適化手法
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# 推定結果

# 指標値計算
root_mse = tf.sqrt(loss_op)

# variableの初期化
init = tf.global_variables_initializer()

get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)

# 計算グラフの実行
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        # エポックごとにループ
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)
        total_loss = 0
        total_rmse = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # ミニバッチごとにループ
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, rmse = sess.run([loss_op, root_mse], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
            total_rmse += rmse
        total_loss /= n_samples
        total_rmse /= n_samples
        val_loss, val_rmse = sess.run([loss_op, root_mse], feed_dict={X: X_val, Y: y_val})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, rmse : {:.3f}, val_rmse : {:.3f}".format(epoch, loss, val_loss, rmse, val_rmse))
    test_rmse = sess.run(root_mse, feed_dict={X: X_test, Y: y_test})
    print("test_rmse : {:.3f}".format(test_rmse))

Epoch 0, loss : 683423104.0000, val_loss : 2598622208.0000, rmse : 26142.361, val_rmse : 50976.684
Epoch 1, loss : 875529536.0000, val_loss : 2457922560.0000, rmse : 29589.348, val_rmse : 49577.441
Epoch 2, loss : 991358336.0000, val_loss : 2437245952.0000, rmse : 31485.844, val_rmse : 49368.473
Epoch 3, loss : 1061776512.0000, val_loss : 2438607104.0000, rmse : 32584.912, val_rmse : 49382.254
Epoch 4, loss : 1094641664.0000, val_loss : 2444333568.0000, rmse : 33085.371, val_rmse : 49440.203
Epoch 5, loss : 1107277952.0000, val_loss : 2447097344.0000, rmse : 33275.785, val_rmse : 49468.145
Epoch 6, loss : 1104332672.0000, val_loss : 2446905600.0000, rmse : 33231.500, val_rmse : 49466.207
Epoch 7, loss : 1100504832.0000, val_loss : 2447325184.0000, rmse : 33173.859, val_rmse : 49470.449
Epoch 8, loss : 1105986432.0000, val_loss : 2448337408.0000, rmse : 33256.375, val_rmse : 49480.676
Epoch 9, loss : 1129820672.0000, val_loss : 2449404672.0000, rmse : 33612.805, val_rmse : 49491.461
tes

### 【問題5】MNISTのモデルを作成 
ニューラルネットワークのスクラッチで使用したMNISTを分類するモデルを作成してください。
3クラス以上の分類という点ではひとつ前のIrisと同様です。入力が画像であるという点で異なります。
スクラッチで実装したモデルの再現を目指してください。

In [4]:
"""
TensorFlowで実装したニューラルネットワークを使いMNISTデータセットを3値分類する
"""
# データセットの読み込み
from keras.datasets import mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# 画像データを2次元に変換
X_train = X_train.reshape(-1, 784)
X_test = X_test.reshape(-1, 784)

X_train = X_train.astype(np.float)
X_test = X_test.astype(np.float)
X_train /= 255
X_test /= 255

#onehot
enc = Oneenc = OneHotEncoder(handle_unknown='ignore', sparse=False)
y_train = enc.fit_transform(y_train[:, np.newaxis])
y_test = enc.fit_transform(y_test[:, np.newaxis])


# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

# ハイパーパラメータの設定
learning_rate = 0.05
batch_size = 100
num_epochs = 10

n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 10

# 計算グラフに渡す引数の形を決める
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])

def example_net(x):
    """
    単純な3層ニューラルネットワーク
    """

    # 重みとバイアスの宣言
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }

    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] # tf.addと+は等価である
    return layer_output

# ネットワーク構造の読み込み                               
logits = example_net(X)

# 目的関数
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=logits))
# 最適化手法
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# 推定結果
correct_pred = tf.equal(tf.argmax(Y,1), tf.argmax(tf.nn.softmax(logits),1))
# 指標値計算
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# variableの初期化
init = tf.global_variables_initializer()

get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)

# 計算グラフの実行
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        # エポックごとにループ
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)
        total_loss = 0
        total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # ミニバッチごとにループ
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
            total_acc += acc
        total_loss /= n_samples
        total_acc /= n_samples
        val_loss, val_acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}, val_acc : {:.3f}".format(epoch, loss, val_loss, acc, val_acc))
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
    print("test_acc : {:.3f}".format(test_acc))

Using TensorFlow backend.


Epoch 0, loss : 1.0929, val_loss : 1.2289, acc : 0.680, val_acc : 0.684
Epoch 1, loss : 0.5300, val_loss : 0.8745, acc : 0.770, val_acc : 0.734
Epoch 2, loss : 0.6028, val_loss : 0.7385, acc : 0.830, val_acc : 0.794
Epoch 3, loss : 0.5071, val_loss : 0.6520, acc : 0.880, val_acc : 0.840
Epoch 4, loss : 0.3914, val_loss : 0.5710, acc : 0.880, val_acc : 0.862
Epoch 5, loss : 0.3673, val_loss : 0.4588, acc : 0.890, val_acc : 0.882
Epoch 6, loss : 0.3187, val_loss : 0.4610, acc : 0.890, val_acc : 0.890
Epoch 7, loss : 0.3286, val_loss : 0.4130, acc : 0.890, val_acc : 0.901
Epoch 8, loss : 0.2919, val_loss : 0.3876, acc : 0.910, val_acc : 0.909
Epoch 9, loss : 0.3330, val_loss : 0.4428, acc : 0.900, val_acc : 0.894
test_acc : 0.894
